## Graded Lab: Reflection in a Research Agent

In this graded lab, you’ll implement a simple **agentic workflow** designed to simulate reflective thinking in a writing task. This is one building block of a more complex research agent that will be constructed throughout the course.

### Objective

Build a three-step workflow where an LLM writes an essay draft, critiques it, and rewrites it. 

* **Step 1 – Drafting:** Call the LLM to generate an initial draft of an essay based on a simple prompt.
* **Step 2 – Reflection:** Reflect on the draft using a reasoning step. (Optionally, this can be done with a different model.)
* **Step 3 – Revision:** Apply the feedback from the reflection to generate a revised version of the essay.


---
<a name='submission'></a>

<h4 style="color:green; font-weight:bold;">TIPS FOR SUCCESSFUL GRADING OF YOUR ASSIGNMENT:</h4>

* All cells are frozen except for the ones where you need to write your solution code or when explicitly mentioned you can interact with it.

* In each exercise cell, look for comments `### START CODE HERE ###` and `### END CODE HERE ###`. These show you where to write the solution code. **Do not add or change any code that is outside these comments**.

* You can add new cells to experiment but these will be omitted by the grader, so don't rely on newly created cells to host your solution code, use the provided places for this.

* Avoid using global variables unless you absolutely have to. The grader tests your code in an isolated environment without running all cells from the top. As a result, global variables may be unavailable when scoring your submission. Global variables that are meant to be used will be defined in UPPERCASE.

* To submit your notebook for grading, first save it by clicking the 💾 icon on the top left of the page and then click on the <span style="background-color: red; color: white; padding: 3px 5px; font-size: 16px; border-radius: 5px;">Submit assignment</span> button on the top right of the page.
---

Before interacting with the language models, we initialize the `aisuite` client. This setup loads environment variables (e.g., API keys) from a `.env` file to securely authenticate with backend services. The `ai.Client()` instance will be used to make all model calls throughout this workflow.

In [14]:
from dotenv import load_dotenv

load_dotenv()

import aisuite as ai

# Define the client. You can use this variable inside your graded functions!
CLIENT = ai.Client()

In [15]:
import unittests

## Exercise 1: `generate_draft` Function

**Objective**:
Write a function called `generate_draft` that takes in a string topic and uses a language model to generate a complete draft essay.

**Inputs**:

* `topic` (str): The essay topic.
* `model` (str, optional): The model identifier to use. Defaults to `"openai:gpt-4o"`.

**Output**:

* A string representing the full draft of the essay.

The setup for calling the LLM using the aisuite library is already provided. Focus on crafting the prompt content. You can reference this setup in later exercises to understand how to interact with the library effectively.


In [16]:
# GRADED FUNCTION: generate_draft

def generate_draft(topic: str, model: str = "openai:gpt-4o") -> str: 
    
    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f"""
        You are an expert essayist and researcher.
        Task: Write a complete, original draft essay on the topic: "{topic}".

        Requirements:
        - Audience: educated general readers; no specialist background assumed.
        - Length: 900–1,200 words.
        - Structure:
          - Title on the first line.
          - Engaging introduction with a clear thesis.
          - 3–5 logically ordered sections with informative subheadings.
          - Conclusion that synthesizes insights and notes implications or next steps.
        - Depth and support:
          - Use specific examples, historical/contextual details, or widely recognized data.
          - If citing facts, attribute to credible sources (e.g., Organization/Author, year); avoid invented quotes or precise stats you’re unsure of.
        - Reasoning:
          - Present a clear throughline of argument.
          - Anticipate one reasonable counterargument and address it.
        - Style:
          - Active voice, precise vocabulary, varied sentence structure.
          - Avoid clichés, filler, and repetition; ensure smooth transitions.
        - If the topic is broad or ambiguous, briefly state reasonable assumptions in 1–2 sentences before the introduction.

        Output: Provide only the final essay with the title and subheadings.
        """

    ### END CODE HERE ###
    
    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
    )

    return response.choices[0].message.content

Run the following cell to check your code is working correctly:

In [17]:
# Test your code!
unittests.test_generate_draft(generate_draft)

 All tests passed!


## Exercise 2: `reflect_on_draft` Function

**Objective**:
Write a function called `reflect_on_draft` that takes a previously generated essay draft and uses a language model to provide constructive feedback.

**Inputs**:

* `draft` (str): The essay text to reflect on.
* `model` (str, optional): The model identifier to use. Defaults to `"openai:o4-mini"`.

**Output**:

* A string with feedback in paragraph form.

**Requirements**:

* The feedback should be critical but constructive.
* It should address issues such as structure, clarity, strength of argument, and writing style.
* The function should send the draft to the model and return its response.

You do **not** need to rewrite the essay at this step—just analyze and reflect on it.


In [18]:
# GRADED FUNCTION: reflect_on_draft

def reflect_on_draft(draft: str, model: str = "openai:o4-mini") -> str:

    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f"""
        You are an experienced writing coach.
        Task: Provide critical but constructive feedback on the following essay draft.

        Essay:
        \"\"\"{draft}\"\"\"

        Instructions:
        - Do not rewrite the essay.
        - Evaluate:
          1) Structure and organization
          2) Clarity and coherence
          3) Strength and logic of argument (evidence, reasoning, counterarguments)
          4) Style and tone (voice, diction, sentence variety)
        - Identify the top 3–5 issues that most affect readability or persuasiveness.
        - For each issue, explain why it matters and give 1–2 concrete, actionable suggestions or examples for improvement.
        - Note specific strengths to retain.
        - If claims need support, point that out and suggest the type of evidence or source (avoid fabricating facts).
        - Keep feedback specific, balanced, and concise.

        Output format:
        - One-line summary
        - Strengths (bulleted)
        - Priority issues (numbered, with suggested fixes)
        - Minor suggestions (bulleted)
        """

    ### END CODE HERE ###

    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
    )

    return response.choices[0].message.content

In [19]:
# Test your code!
unittests.test_reflect_on_draft(reflect_on_draft)

 All tests passed!


## Exercise 3: `revise_draft` Function

**Objective**:
Implement a function called `revise_draft` that improves a given essay draft based on feedback from a reflection step.

**Inputs**:

* `original_draft` (str): The initial version of the essay.
* `reflection` (str): Constructive feedback or critique on the draft.
* `model` (str, optional): The model identifier to use. Defaults to `"openai:gpt-4o"`.

**Output**:

* A string containing the revised and improved essay.

**Requirements**:

* The revised draft should address the issues mentioned in the feedback.
* It should improve clarity, coherence, argument strength, and overall flow.
* The function should use the feedback to guide the revision, and return only the final revised essay.

In this final exercise, you'll also need to manage the call to the LLM using the CLIENT, as you've practiced in previous exercises.

In [20]:
# GRADED FUNCTION: revise_draft

def revise_draft(original_draft: str, reflection: str, model: str = "openai:gpt-4o") -> str:

    ### START CODE HERE ###

    # Define your prompt here. A multi-line f-string is typically used for this.
    prompt = f"""
        You are an expert essay editor and rewriter.

        Inputs:
        Original draft:
        \"\"\"{original_draft}\"\"\"

        Feedback to address:
        \"\"\"{reflection}\"\"\"

        Task:
        Revise the essay to incorporate the feedback while preserving the original intent, thesis, and any accurate facts.

        Requirements:
        - Address every actionable point in the feedback; if trade-offs arise, prioritize clarity, coherence, and argument strength.
        - Improve:
          • Thesis focus and section flow
          • Logical progression and transitions
          • Evidence and reasoning (tighten claims; suggest [citation needed] rather than invent specifics)
          • Style and tone: active voice, concise sentences, varied structure
        - Remove redundancy and filler; resolve contradictions; fix grammar and punctuation.
        - Maintain approximately the original length (±10%) unless the feedback specifies otherwise.
        - Preserve the author’s voice where possible while elevating precision and readability.
        - If the feedback requests structural changes (e.g., add subheadings, reorganize sections), implement them.

        Constraints:
        - Do not fabricate quotes, statistics, or sources.
        - Do not include analysis or an explanation of edits.

        Output:
        Provide only the revised essay (use a clear title if the original had one or the feedback requests it).
        """

    # Get a response from the LLM by creating a chat with the client.
    response = CLIENT.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
    )

    ### END CODE HERE ###

    return response.choices[0].message.content

In [21]:
# Test your code!
unittests.test_revise_draft(revise_draft)

 All tests passed!


### 🧪 Test the Reflective Writing Workflow

Use the functions you implemented to simulate the complete writing workflow:

1. **Generate a draft** in response to the essay prompt.
2. **Reflect** on the draft to identify improvements.
3. **Revise** the draft using the feedback.

Observe the outputs of each step. You do **not** need to modify the outputs — just verify that the workflow runs as expected and each component returns a valid string.


In [22]:
essay_prompt = "Should social media platforms be regulated by the government?"

# Agent 1 – Draft
draft = generate_draft(essay_prompt)
print("📝 Draft:\n")
print(draft)

# Agent 2 – Reflection
feedback = reflect_on_draft(draft)
print("\n🧠 Feedback:\n")
print(feedback)

# Agent 3 – Revision
revised = revise_draft(draft, feedback)
print("\n✍️ Revised:\n")
print(revised)

📝 Draft:

Should Social Media Platforms Be Regulated by the Government?

In the last two decades, social media has transformed from a nascent technology to an omnipresent force that shapes global communication, commerce, and culture. Platforms like Facebook, Twitter, and Instagram are not just tools for socializing; they influence political elections, revolutionize marketing, and even play roles in geopolitics. Amidst their growing influence, a pressing question has emerged: should social media platforms be regulated by the government? While these platforms provide unprecedented avenues for free expression, the potential for abuse and harm alongside profound societal impacts makes government regulation a compelling consideration. In this essay, I will argue that while regulation is necessary to prevent harm and foster accountability, it should be balanced to protect freedom of expression and innovation.

The Case for Regulation

The primary argument for regulating social media platform


✍️ Revised:

Should Social Media Platforms Be Regulated by the Government?

In the past two decades, social media has evolved from a fledgling innovation to a dominant influence in global communication, commerce, and culture. Platforms such as Facebook, Twitter, and Instagram are now integral not only to personal interaction but also to political processes, marketing, and even international relations. Given their significant impact, the question arises: should governments regulate social media platforms? These platforms offer unparalleled opportunities for free expression, yet the potential for misuse and their profound societal effects warrant serious consideration of regulation. This essay argues that while regulation is essential to prevent harm and ensure accountability, it must be carefully crafted to protect freedom of expression and foster innovation.

The Case for Regulation

Regulation of social media platforms is crucial to mitigating harm. These platforms can spread misinfo

To better visualize the output of each step in the reflective writing workflow, we use a utility function called `show_output`. This function displays the results of each stage (drafting, reflection, and revision) in styled boxes with custom background and text colors, making it easier to compare and understand the progression of the essay.


In [23]:
from utils import show_output

essay_prompt = "Should social media platforms be regulated by the government?"

show_output("Step 1 – Draft", draft, background="#fff8dc", text_color="#333333")
show_output("Step 2 – Reflection", feedback, background="#e0f7fa", text_color="#222222")
show_output("Step 3 – Revision", revised, background="#f3e5f5", text_color="#222222")

## Check grading feedback

If you have collapsed the right panel to have more screen space for your code, as shown below:

<img src="./images/collapsed.png" alt="Collapsed Image" width="800" height="400"/>

You can click on the left-facing arrow button (highlighted in red) to view feedback for your submission after submitting it for grading. Once expanded, it should display like this:

<img src="./images/expanded.png" alt="Expanded Image" width="800" height="400"/>